# **Model Spare Part Price Prediction** 






\TO-DO 08.06.22:
-Grid Search auf 12 Features eingrenzen
-Features einzeln benennen --> Features_1
-Best Hyperparameter aus der Grid Search übernehmen
-Evaluierung des Models: Nested Cross Validation

**Overview:**

1. Loading the data
2. Partially clean the data, partially the data has already been cleaned in Excel
3. Visualise data to identify correlations
4. Split into test and training data sets
5. Creation of the model
6. Training the model
7. Testing and validating the model 
8. Output 

### Import relevant libaries



 

In [5]:
pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [6]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
import os
import mpl_toolkits
import param as pp
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import datetime
%matplotlib inline
import io

sns.set()

import os
import pandas as pd
import anvil.server
import anvil.media
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
# get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from math import sqrt
import mpl_toolkits
import param as pp
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold
import os
from sklearn.model_selection import train_test_split

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go


In [18]:
def train_eval(features_col):
  train = pd.read_excel("/content/train data.xlsx")
  inference_dataset = pd.read_excel("/content/test data.xlsx")
  inference_dataset.drop(columns=['ID'],axis=1,inplace=True) 
  train.drop(columns=['ID'],axis=1,inplace=True) 

  ep = train['Länge in mm'].value_counts().reset_index()
  ep.columns = [
      'Länge', 
      'percent'
  ]
  ep['percent'] /= len(train)


  ep = train['Biegungen'].value_counts().reset_index()
  ep.columns = [
      'Biegungen', 
      'percent'
  ]
  ep['percent'] /= len(train)

  labels = {}
  for col in train.select_dtypes(exclude = np.number).columns.tolist():
      le = LabelEncoder().fit(pd.concat([train[col].astype(str),inference_dataset[col].astype(str)]))   
      train[col] = le.transform(train[col].astype(str))
      inference_dataset[col] = le.transform(inference_dataset[col].astype(str))
      labels [col] = le
  print('Categorical columns:', list(labels.keys()))



  print(f'Percent of Nans in Train Data : {round(train.isna().sum().sum()/len(train), 2)}')
  print(f'Percent of Nans in Test  Data : {round(inference_dataset.isna().sum().sum()/len(inference_dataset), 2)}')

  train = train.replace([np.inf, -np.inf], np.nan)
  train= train.fillna(train.mean())


  inference_dataset = inference_dataset.replace([np.inf, -np.inf], np.nan)
  inference_dataset= inference_dataset.fillna(inference_dataset.mean())
  if features_col == '': 
    features = train.drop(columns=['Preis'],axis=1).columns
    # features =['Nettogewicht','Länge in mm','Nirosta','Stärke','Biegungen','Durchmesser_innen','Fertigungsherkunft',
    #       'Wettbewerbsintensität', 'Komplexität', 'Nutzungsdauer', 'Militärische Verwendung', 'Baureihe'
    # ]
  else:
    features = features_col
  target = 'Preis'

  Y = train['Preis']
  X = train.drop(columns=['Preis'])
  X_train, X_test, Y_train, Y_test = train_test_split(X[features], Y, test_size=0.15, random_state=9)


  print('X train shape: ', X_train.shape)
  print('Y train shape: ', Y_train.shape)
  print('X test shape: ', X_test.shape)
  print('Y test shape: ', Y_test.shape)
  # X[features].head()
  print(features)


  # We define the model
  estimator = RandomForestRegressor(random_state = 42,criterion='squared_error')
  para_grids = {
              "n_estimators" : [10,50,100,200],
              #"n_estimators" : [10,50,100],
              "max_features" : ["auto", "log2", "sqrt"],
              'max_depth' : [1,2,3,4,5,6,7,8,9,10],
              #'max_depth' : [3,5,7,9,11,13],
              "bootstrap"    : [True, False]
          }
  # para_grids = {
  #           "n_estimators" : [10,50],
  #           #"n_estimators" : [10,50],
  #           # "max_features" : ["auto", "log2", "sqrt"],
  #           'max_depth' : [3,4],
  #           # 'max_depth' : [3,5,7,9,11,13],
  #           # "bootstrap"    : [True, False]
  #       }

  inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
  outer_cv = KFold(n_splits=3, shuffle=True, random_state=42)

  # Non_nested parameter search and scoring
  clf = GridSearchCV(estimator=estimator, param_grid=para_grids, cv=outer_cv)
  clf.fit(train[features], train[target])
  non_nested_score = clf.best_score_
  print('Non Nested Score: %.3f' % (non_nested_score))
  best_param = clf.best_estimator_
  print(best_param)

  # Nested CV with parameter optimization
  clf = GridSearchCV(estimator=estimator, param_grid=para_grids, cv=inner_cv)
  n_score = cross_val_score(clf, X=train[features], y=train[target], cv=outer_cv)
  nested_score = n_score.mean()
  # The following is the most realistic performance estimate for the model selection pipeline!
  print('Nested Score: %.3f' % (nested_score))
  score_difference = non_nested_score - nested_score
  print('Score Difference: %.3f' % (score_difference))

  # Nested CV with parameter optimization to yield predictions on the entire training set
  clf = GridSearchCV(estimator=estimator, param_grid=para_grids, cv=inner_cv)
  predictions = cross_val_predict(clf, X=train[features], y=train[target], cv=outer_cv)
  train['Preis_Predicted']= predictions

  train.to_excel("output.xlsx", sheet_name='Output_Hydraulikleitungen') 

  mean = "Mean Squared Error: " + str(sqrt(mean_squared_error(train.Preis, train.Preis_Predicted))) + " \n"  
  msr   =        "Mean Squared Error:  " + str(sqrt(mean_squared_error(train.Preis, train.Preis_Predicted)))  + " \n"
  mar    =    "Mean Absolute Error:  "+ str(mean_absolute_error(train.Preis, train.Preis_Predicted)) + " \n"
  mar2    = "Median Absolute Error:  "+ str(median_absolute_error(train.Preis, train.Preis_Predicted)) + " \n"
  r2   = "R2 Score: "+ str(r2_score(train.Preis, train.Preis_Predicted))
  result = mean + msr + mar + mar2 + r2  
  params = {"n_estimators":best_param.n_estimators ,
            "max_features":best_param.max_features,
            "bootstrap":best_param.bootstrap,
            "max_depth":best_param.max_depth}
  print('Mean Squared Error: %.3f' % (mean_squared_error(train.Preis, train.Preis_Predicted)))

  print('Mean Squared Error: %.3f' % sqrt(mean_squared_error(train.Preis, train.Preis_Predicted)))
  print('Mean Absolute Error: %.3f' % (mean_absolute_error(train.Preis, train.Preis_Predicted)))
  print('Median Absolute Error: %.3f' % (median_absolute_error(train.Preis, train.Preis_Predicted)))
  print('R2 Score: %.3f' % (r2_score(train.Preis, train.Preis_Predicted)))

  inference_predictions = best_param.predict(inference_dataset[features])


  features =['Länge in mm','Nirosta','Stärke','Biegungen','Durchmesser_innen','Fertigungsherkunft',
        'Wettbewerbsintensität', 'Komplexität', 'Nutzungsdauer', 'Militärische Verwendung', 'Baureihe']
  submission = pd.DataFrame({'ID':inference_dataset['ID'],'Länge in mm':inference_dataset['Länge in mm'],'Nettogewicht':inference_dataset['Nettogewicht'],'Biegungen':inference_dataset['Biegungen'],'Durchmesser_innen':inference_dataset['Durchmesser_innen'],'Komplexität':inference_dataset['Komplexität'],'Militärische Verwendung':inference_dataset['Militärische Verwendung'],'Preis':inference_predictions})                        


  filename = 'submission_hydraulic_pipe1.csv'

  submission.to_csv(filename,index=True)


  print('Saved file: ' + filename)
  return result,params

In [19]:
def train_withparams(features_col='',n_estimators=100,max_depth=4,max_features ="auto",bootstrap=True, random_state=42):
  train = pd.read_excel("/content/train data.xlsx")
  inference_dataset = pd.read_excel("/content/test data.xlsx")
 
  ep = train['Länge in mm'].value_counts().reset_index()
  ep.columns = [
      'Länge', 
      'percent'
  ]
  ep['percent'] /= len(train)


  ep = train['Biegungen'].value_counts().reset_index()
  ep.columns = [
      'Biegungen', 
      'percent'
  ]
  ep['percent'] /= len(train)

  labels = {}
  for col in train.select_dtypes(exclude = np.number).columns.tolist():
      le = LabelEncoder().fit(pd.concat([train[col].astype(str),inference_dataset[col].astype(str)]))   
      train[col] = le.transform(train[col].astype(str))
      inference_dataset[col] = le.transform(inference_dataset[col].astype(str))
      labels [col] = le
  print('Categorical columns:', list(labels.keys()))



  print(f'Percent of Nans in Train Data : {round(train.isna().sum().sum()/len(train), 2)}')
  print(f'Percent of Nans in Test  Data : {round(inference_dataset.isna().sum().sum()/len(inference_dataset), 2)}')

  train = train.replace([np.inf, -np.inf], np.nan)
  train= train.fillna(train.mean())


  inference_dataset = inference_dataset.replace([np.inf, -np.inf], np.nan)
  inference_dataset= inference_dataset.fillna(inference_dataset.mean())

  if features_col == '': 
    features = train.drop(columns=['Preis'],axis=1).columns
    # features =['Nettogewicht','Länge in mm','Nirosta','Stärke','Biegungen','Durchmesser_innen','Fertigungsherkunft',
    #       'Wettbewerbsintensität', 'Komplexität', 'Nutzungsdauer', 'Militärische Verwendung', 'Baureihe'
    # ]
  else:
   features = features_col
  target = 'Preis'

  Y = train['Preis']
  X = train.drop(columns=['Preis'])
  X_train, X_test, Y_train, Y_test = train_test_split(X[features], Y, test_size=0.15, random_state=random_state)

  print('X train shape: ', X_train.shape)
  print('Y train shape: ', Y_train.shape)
  print('X test shape: ', X_test.shape)
  print('Y test shape: ', Y_test.shape)
  # X[features].head()
  print(features)


  # We define the model
  estimator = RandomForestRegressor( max_depth=max_depth,max_features=max_features  , n_estimators=n_estimators,bootstrap=bootstrap, random_state = 42,criterion='squared_error')
  estimator.fit(train[features], train[target])

  
  predictions = estimator.predict(train[features])
  train['Preis_Predicted']= predictions

  train.to_excel("output.xlsx", sheet_name='Output_Hydraulikleitungen') 

  mean = "Mean Squared Error: " + str(sqrt(mean_squared_error(train.Preis, train.Preis_Predicted))) + " \n"  
  msr   =        "Mean Squared Error:  " + str(sqrt(mean_squared_error(train.Preis, train.Preis_Predicted)))  + " \n"
  mar    =    "Mean Absolute Error:  "+ str(mean_absolute_error(train.Preis, train.Preis_Predicted)) + " \n"
  mar2    = "Median Absolute Error:  "+ str(median_absolute_error(train.Preis, train.Preis_Predicted)) + " \n"
  r2   = "R2 Score: "+ str(r2_score(train.Preis, train.Preis_Predicted))
  result = mean + msr + mar + mar2 + r2  
  

  inference_predictions = estimator.predict(inference_dataset[features])

  features =['Länge in mm','Nirosta','Stärke','Biegungen','Durchmesser_innen','Fertigungsherkunft',
        'Wettbewerbsintensität', 'Komplexität', 'Nutzungsdauer', 'Militärische Verwendung', 'Baureihe']
  submission = pd.DataFrame({'ID':inference_dataset['ID'],'Länge in mm':inference_dataset['Länge in mm'],'Nettogewicht':inference_dataset['Nettogewicht'],'Biegungen':inference_dataset['Biegungen'],'Durchmesser_innen':inference_dataset['Durchmesser_innen'],'Komplexität':inference_dataset['Komplexität'],'Militärische Verwendung':inference_dataset['Militärische Verwendung'],'Preis':inference_predictions})                        


  filename = 'submission_hydraulic_pipe1.csv'

  submission.to_csv(filename,index=True)


  print('Saved file: ' + filename)
  return result

In [22]:
# train_withparams()
# pd.read_excel('train data.xlsx').drop(columns=['ID'],axis=1,inplace=True) 
# pd.read_excel("/content/train data.xlsx")['Wert Nirosta']

In [20]:

anvil.server.connect("server_FPS4H4JV6LL2OEIZMQY4M66E-7VNAX7SHYSCBEHWO")

@anvil.server.callable
def say_hello(name):
  print("Hello from the uplink, %s!" % name)
# anvil.server.wait_forever()


@anvil.server.callable
def calculate_model_withparam(features, n_estimators,max_depth,max_features ,bootstrap):
  res = train_withparams(features,n_estimators,max_depth,max_features ,bootstrap)
  df1 = pd.read_csv("/content/submission_hydraulic_pipe1.csv")
  content = io.BytesIO()
  df1.to_csv(content, index=False)
  content.seek(0, 0)
  return anvil.BlobMedia(content=content.read(), content_type="text/csv",name='output.csv'), res
  # csv_media = anvil.BlobMedia("text/plain", df1, name="Report.csv")
  # return 
@anvil.server.callable
def calculate_model(features_col):
  res,params = train_eval(features_col)
  df1 = pd.read_csv("/content/submission_hydraulic_pipe1.csv")
  content = io.BytesIO()
  df1.to_csv(content, index=False)
  content.seek(0, 0)
  return anvil.BlobMedia(content=content.read(), content_type="text/csv",name='output.csv'), res,params
  # csv_media = anvil.BlobMedia("text/plain", df1, name="Report.csv")
  # return 
i = 0
@anvil.server.callable
def check_files():
  df1 = ""
  try:
    df1 = pd.read_excel("/content/train data.xlsx")
  except:
    return "no"
  content = io.BytesIO()
  df1.to_csv(content, index=False)
  content.seek(0, 0)
  return anvil.BlobMedia(content=content.read(), content_type="text/csv",name='output.csv')

@anvil.server.callable
def upload_excel_data(file,name):
  # for i,file in enumerate(files):
  with open(name, "wb") as f:
    f.write(file.get_bytes())
    # df = pd.read_csv(f)
  return  str(i)
def download_out():
  df1 = pd.read_excel("/content/ss.xlsx")
  content = io.BytesIO()
  df1.to_excel(content, index=False)
  content.seek(0, 0)
  return anvil.BlobMedia(content=content.read(), content_type="application/vnd.ms-excel",name='output.xlsx')
anvil.server.wait_forever()

Categorical columns: ['Fertigungsherkunft', 'Baureihe Datum']
Percent of Nans in Train Data : 0.1
Percent of Nans in Test  Data : 1.38
X train shape:  (211, 24)
Y train shape:  (211,)
X test shape:  (38, 24)
Y test shape:  (38,)
Index(['Nettogewicht', 'Länge in mm', 'Länge', 'Nirosta', 'Wert Nirosta',
       'Stärke', 'Biegungen', 'Wert Biegungen', 'Durchmesser_aussen',
       'Wert Durchmesser', 'Durchmesser_innen', 'Fertigungsherkunft',
       'Wert Fertigungsherkunft', 'Wettbewerbsintensität',
       'Wert Wettbewerbsintensität', 'Komplexität', 'Wert Komplexität',
       'Nutzungsdauer', 'Militärische Verwendung',
       'Wert Militärische Verwendung', 'Baureihe Datum', 'Baureihe',
       'Wert Baureihe', 'Value'],
      dtype='object')


<ipython-input-18-2173546dfd47>:40: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inference_dataset= inference_dataset.fillna(inference_dataset.mean())


Non Nested Score: 0.949
RandomForestRegressor(max_depth=4, n_estimators=50, random_state=42)
Nested Score: 0.946
Score Difference: 0.003
Mean Squared Error: 162.059
Mean Squared Error: 12.730
Mean Absolute Error: 8.571
Median Absolute Error: 6.159
R2 Score: 0.947


KeyboardInterrupt: ignored